### Materials Project Workshop – July 28th to July 30th, 2019, Berkeley, California

![pymatgen logo](http://pymatgen.org/_images/pymatgen.png)

# 0. What is pymatgen?

Pymatgen (Python Materials Genomics) is the code that powers all of the scientific analysis behind the Materials Project.  It includes a robust and efficient libraries for the handling of crystallographic structures and molecules, in addition to various mathematical and scientific tools for the handling and generation of materials data. For more details, see the [pymatgen website](https://pymatgen.org).

## 0.1 How do I install pymatgen?

For the workshop, pymatgen has been pre-installed for use in your Jupyter notebooks.

Otherwise, pymatgen can be installed via pip:

`pip install pymatgen`

or conda:

`conda install --channel matsci pymatgen`

We recommend using Python 3.6 or above. Until 2018, pymatgen was developed simultaneously for Python 2.x and 3.x, but following the rest of the Python community we have phased out support for Python 2.x, and since version 2019.1.1 we are developing exclusively for Python 3.x.

## 0.2 Where can I find help and how do I get involved?

* **For general help:** [pymatgen discourse](https://pymatgen.discourse.group/) is a place to ask questions.

* **To report bugs:** The [Github Issues](https://github.com/materialsproject/pymatgen/issues) page is a good place to report bugs.

* **For Materials Project data and website discussions:** The Materials Project  has its community [Materials Project Discussion](https://discuss.materialsproject.org) forum. 

* **For more example notebooks:** [matgenb](http://matgenb.materialsvirtuallab.org) is a new resource of Jupyter notebooks demonstrating various pymatgen functionality.

If you want specific new features, you're welcome to ask! We try to respond to community needs. If you're a developer and can add the feature yourself, we actively encourage you to do so by creating a Pull Request on Github with your additional functionality. To date, pymatgen has seen over 17,000 commits and over 120 contributors, and we try to have an inclusive and welcoming development community. All contributors are also individually acknowledged on [materialsproject.org/about](https://materialsproject.org/about).


# 1. Verify we have pymatgen installed

First, let's verify we have pymatgen installed. The following command should produce no error or warning:

In [14]:
import pymatgen

We'll also import the data file for the workshop, and the Python os library to manipulate paths

In [15]:
import os
from mp_workshop.data import data_dir

We can show the specific version of pymatgen installed:

In [16]:
print(pymatgen.__version__)

2020.1.10


For a list of new features, bug fixes and other changes, consult the [changelog on pymatgen.org](http://pymatgen.org/change_log.html).

You can also see where pymatgen is installed on your computer:

In [17]:
print(pymatgen.__file__)

/Users/ewcss/software/pymatgen/pymatgen/__init__.py


We can also see which version of the Python programming language we are using:

In [18]:
import sys
print(sys.version)

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


If you have problems or need to report bugs when using pymatgen after the workshop, the above information is often very useful to help us identify the problem.

# 2. Structures and Molecules

Most of the fundamentals of pymatgen are expressed in terms of [**`Molecule`**](http://pymatgen.org/pymatgen.core.structure.html#pymatgen.core.structure.Molecule) and [**`Structure`**](http://pymatgen.org/pymatgen.core.structure.html#pymatgen.core.structure.Structure) objects.

While we will mostly be using `Structure`, `Stucture` and `Molecule` are very similar conceptually. The main difference is that `Structure` supports full periodicity required to describe crystallographic structures.

Creating a `Structure` can be done in one line, even for complicated crystallographic structures. However, we'll start by introducing the somewhat simpler `Molecule` object, and then use this understanding of `Molecule` to introduce `Structure`.

## 2.0.0 Creating a Molecule

Start by importing `Molecule`:

In [19]:
from pymatgen import Molecule

In a Jupyter notebook, you can show help for any Python object by clicking on the object and pressing **Shift+Tab**. This will give you a list of arguments and keyword arguments necessary to construct the object, as well as the documentation ('docstring') which gives more information on what each argument means.

In [20]:
Molecule

pymatgen.core.structure.Molecule

Molecule takes input **arguments** `species` and `coords`, and input **keyword arguments** `charge`, `spin_multiplicity`, `validate_proximity` and `site_properties`.

Keyword arguments come with a default value (the value after the equals sign), and so keyword arguments are optional.

Arguments (without default values) are mandatory.

In [21]:
c_monox = Molecule(["C","O"], [[0.0, 0.0, 0.0], [0.0, 0.0, 1.2]])
print(c_monox)

Full Formula (C1 O1)
Reduced Formula: CO
Charge = 0.0, Spin Mult = 1
Sites (2)
0 C     0.000000     0.000000     0.000000
1 O     0.000000     0.000000     1.200000


Alright, now let's change one of those default variables. How about we make an anion?

In [22]:
oh_minus = Molecule(["O", "H"], [[0.0, 0.0, 0.0], [0.0, 0.0, 1.0]], charge=-1)
print(oh_minus)

Full Formula (H1 O1)
Reduced Formula: H2O2
Charge = -1, Spin Mult = 1
Sites (2)
0 O     0.000000     0.000000     0.000000
1 H     0.000000     0.000000     1.000000


You can also create Molecule objects from files. Let's say you have an \*.xyz file called "water.xyz". You can import that into pymatgen with `Molecule.from_file`, like:

In [23]:
water_file = os.path.join(data_dir, "pymatgen", "water.xyz")
water = Molecule.from_file(water_file)
print(water)

Full Formula (H2 O1)
Reduced Formula: H2O
Charge = 0, Spin Mult = 1
Sites (3)
0 O    -0.070000    -0.026960    -0.095240
1 H     0.919330    -0.015310    -0.054070
2 H    -0.359290     0.231000     0.816010


## 2.0.2 Visualizing a Molecule

It can be very helpful to quickly visualize molecules or structures. We've developed the [Crystal Toolkit](https://github.com/materialsproject/crystaltoolkit) library, which will allow us to automatically view Molecules and Structures within Jupyter notebooks.

To properly visualize Molecule objects, including their bonds, it is necessary to first create a [**`MoleculeGraph`**](http://pymatgen.org/pymatgen.analysis.graphs.html#pymatgen.analysis.graphs.MoleculeGraph) object. A MoleculeGraph contains a Molecule but also holds information about the connections (typically bonds) within that Molecule. Here, we use a local environment class ([**`MinimumDistanceNN`**](http://pymatgen.org/pymatgen.analysis.local_env.html#pymatgen.analysis.local_env.MinimumDistanceNN) or [**`CovalentBondNN`**](http://pymatgen.org/pymatgen.analysis.local_env.html#pymatgen.analysis.local_env.CovalentBondNN)) to automatically determine the bonding environment.

As we'll see, visualizing Structures is even easier than visualizing Molecules.

In [24]:
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import CovalentBondNN, MinimumDistanceNN

import crystal_toolkit

In [25]:
water_mg = MoleculeGraph.with_local_env_strategy(water, CovalentBondNN())
water_mg

Molecule Graph
Molecule: 
Molecule Summary
Site: O (-0.0700, -0.0270, -0.0952)
Site: H (0.9193, -0.0153, -0.0541)
Site: H (-0.3593, 0.2310, 0.8160)
Graph: bonds
from    to  to_image      weight
----  ----  ------------  ------------------
   0     1  (0, 0, 0)     8.424e-01
   0     2  (0, 0, 0)     8.424e-01



Molecule Graph
Molecule: 
Molecule Summary
Site: O (-0.0700, -0.0270, -0.0952)
Site: H (0.9193, -0.0153, -0.0541)
Site: H (-0.3593, 0.2310, 0.8160)
Graph: bonds
from    to  to_image      weight
----  ----  ------------  ------------------
   0     1  (0, 0, 0)     8.424e-01
   0     2  (0, 0, 0)     8.424e-01


## 2.0.3 Exercise

Try it yourself! Create molecules however you like!

In this folder are several example molecules (`methane.xyz`, `furan.xyz`, and `benzene.xyz`). Try loading these files with `Molecule.from_file`. You can also try making a Molecule from a list of species and coordinates. Try changing the default parameters - see what you can and cannot do.

Once you've made your molecule, visualize it to make sure it looks as you'd expect.

## 2.1.0 What's in a Molecule? Introducing Sites, Elements and Species

You can access properties of the molecule, such as the Cartesian coordinates of its sites:

In [26]:
print(c_monox.cart_coords)

[[0.  0.  0. ]
 [0.  0.  1.2]]


or properties that are computed on-the-fly, such as its center of mass:

In [27]:
print(c_monox.center_of_mass)

[0.         0.         0.68544132]


To see the full list of available properties and methods, press **Tab** after typing `my_molecule.` in your Jupyter notebook. There are methods used to modify the molecule and these take additional argument(s). For example, to add a charge to the molecule:

In [28]:
c_monox.set_charge_and_spin(charge=1)

In [29]:
print(c_monox)

Full Formula (C1 O1)
Reduced Formula: CO
Charge = 1, Spin Mult = 2
Sites (2)
0 C     0.000000     0.000000     0.000000
1 O     0.000000     0.000000     1.200000


A molecule is essentially a list of `Site` objects. We can access these sites like we would a list in Python. For example, to obtain the total number of sites in the molecule:

In [30]:
len(c_monox)

2

Or to access the first site (note that Python is a 0-indexed programming language, so the first site is site 0):

In [31]:
print(c_monox[0])

[0. 0. 0.] C


And just like a list, I can even change the elements of a molecule.

In [32]:
c_monox[0] = "O"
c_monox[1] = "C"
c_monox

Molecule Summary
Site: O (0.0000, 0.0000, 0.0000)
Site: C (0.0000, 0.0000, 1.2000)


Molecule Summary
Site: O (0.0000, 0.0000, 0.0000)
Site: C (0.0000, 0.0000, 1.2000)

A site object contains information on the site's identity and position in space.

In [33]:
site0 = c_monox[0]

In [34]:
site0.coords

array([0., 0., 0.])

In [35]:
site0.specie

Element O

Here, because we switched the elements, the site holds the element O. In general, a site can hold an [**`Element`**](http://pymatgen.org/pymatgen.core.periodic_table.html#pymatgen.core.periodic_table.Element), a [**`Specie`**](http://pymatgen.org/pymatgen.core.periodic_table.html#pymatgen.core.periodic_table.Specie) or a [**`Composition`**](http://pymatgen.org/pymatgen.core.composition.html#pymatgen.core.composition.Composition). Let's look at each of these in turn.

In [36]:
from pymatgen import Element, Specie, Composition

An `Element` is simply an element from the Periodic Table.

In [37]:
carbon = Element('C')

Elements have properties such as atomic mass, average ionic radius and more:

In [38]:
carbon.average_ionic_radius

0.3

A `Specie` can contain additional information, such as oxidation state:

In [39]:
o_ion = Specie('O', oxidation_state=-2)
o_ion

Specie O2-


Specie O2-

Again, we can access both these **`Specie`**-specific properties and more general properties of Elements.

In [40]:
o_ion.oxi_state

-2

In [41]:
o_ion.atomic_mass

15.9994

Or, for convenience:

In [42]:
Specie.from_string('O2-')

Specie O2-


Specie O2-

Finally, a `Composition` is an object that can hold certain amounts of different elements or species. This is most useful in a disordered Structure, and would rarely be used in a Molecule. For example, a site that holds 50% Au and 50% Cu would be set as follows:

In [43]:
comp = Composition({'Au': 0.5, 'Cu': 0.5})

A **`Composition`** contains more information than either an **`Element`** or a **`Specie`**. Because it can contain multiple **`Elements`**/**`Species`**, you can obtain the formula, or the chemical system.

In [44]:
print("formula", comp.alphabetical_formula)
print("chemical system", comp.chemical_system)

formula Au0.5 Cu0.5
chemical system Au-Cu


When we construct a `Molecule`, the input argument will automatically be converted into one of `Element`, `Specie` or `Composition`. Thus, in the previous example, when we first defined carbon monoxide, the input `['C', 'O']` was converted to `[Element C, Element O]`.

## 2.1.1 Exercise: Hydrogen Cyanide

Construct the linear HCN molecule where each bond distance is the sum of the two atomic radii. The atomic spheres should not substantially overlap.

In [45]:
H_rad = Element('H').atomic_radius
C_rad = Element('C').atomic_radius
N_rad = Element('N').atomic_radius
HC_bond_dist = H_rad + C_rad
CN_bond_dist = C_rad + N_rad
H_pos = 0
C_pos = H_pos + HC_bond_dist
N_pos = C_pos + CN_bond_dist
hcn = Molecule(['H','C','N'], [[H_pos, 0, 0], [C_pos, 0, 0],[N_pos, 0, 0]])
hcn_mg = MoleculeGraph.with_local_env_strategy(hcn, CovalentBondNN())
hcn_mg

Molecule Graph
Molecule: 
Molecule Summary
Site: H (0.0000, 0.0000, 0.0000)
Site: C (0.9500, 0.0000, 0.0000)
Site: N (2.3000, 0.0000, 0.0000)
Graph: bonds
from    to  to_image      weight
----  ----  ------------  ------------------
   0     1  (0, 0, 0)     1.000e+00
   1     2  (0, 0, 0)     1.706e+00



Molecule Graph
Molecule: 
Molecule Summary
Site: H (0.0000, 0.0000, 0.0000)
Site: C (0.9500, 0.0000, 0.0000)
Site: N (2.3000, 0.0000, 0.0000)
Graph: bonds
from    to  to_image      weight
----  ----  ------------  ------------------
   0     1  (0, 0, 0)     1.000e+00
   1     2  (0, 0, 0)     1.706e+00


## 2.2.0 Creating a Structure and Lattice

Creating a `Structure` is very similar to creating a `Molecule`, except we now also have to specify a `Lattice`. 

In [46]:
from pymatgen import Structure, Lattice

A `Lattice` can be created in one of several ways, such as by inputting a 3x3 matrix describing the individual lattice vectors. For example, a cubic lattice of length 5 Ångstrom:

In [47]:
my_lattice = Lattice([[5, 0, 0], [0, 5, 0], [0, 0, 5]])

In [48]:
my_lattice

Lattice
    abc : 5.0 5.0 5.0
 angles : 90.0 90.0 90.0
 volume : 125.0
      A : 5.0 0.0 0.0
      B : 0.0 5.0 0.0
      C : 0.0 0.0 5.0


Lattice
    abc : 5.0 5.0 5.0
 angles : 90.0 90.0 90.0
 volume : 125.0
      A : 5.0 0.0 0.0
      B : 0.0 5.0 0.0
      C : 0.0 0.0 5.0

Equivalently, we can create it from its lattice parameters:

In [49]:
my_lattice_2 = Lattice.from_parameters(5, 5, 5, 90, 90, 90)

Or, since we know in this case that we have a cubic lattice, a == b == c and alpha == beta == gamma == 90 degrees, so we can simply put:

In [50]:
my_lattice_3 = Lattice.cubic(5)

We can confirm that these lattices are the same:

In [51]:
my_lattice == my_lattice_2 == my_lattice_3

True

Now, we can create a simple crystal structure. Let's start with body-centered-cubic iron:

In [52]:
bcc_fe = Structure(Lattice.cubic(2.8), ["Fe", "Fe"], [[0, 0, 0], [0.5, 0.5, 0.5]])
bcc_fe

Structure Summary
Lattice
    abc : 2.8 2.8 2.8
 angles : 90.0 90.0 90.0
 volume : 21.951999999999995
      A : 2.8 0.0 0.0
      B : 0.0 2.8 0.0
      C : 0.0 0.0 2.8
PeriodicSite: Fe (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Fe (1.4000, 1.4000, 1.4000) [0.5000, 0.5000, 0.5000]


Structure Summary
Lattice
    abc : 2.8 2.8 2.8
 angles : 90.0 90.0 90.0
 volume : 21.951999999999995
      A : 2.8 0.0 0.0
      B : 0.0 2.8 0.0
      C : 0.0 0.0 2.8
PeriodicSite: Fe (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Fe (1.4000, 1.4000, 1.4000) [0.5000, 0.5000, 0.5000]

In [53]:
print(bcc_fe)

Full Formula (Fe2)
Reduced Formula: Fe
abc   :   2.800000   2.800000   2.800000
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0.5  0.5  0.5


Creating this `Structure` was similar to creating a `Molecule` in that we provided a list of elements and a list of positions. However, there are two key differences: we had to include our `Lattice` object when creating the `Structure` and since we have a lattice, we can define the positions of our sites in *fractional coordinates* with respect to that lattice instead of Cartesian coordinates.

It's also possible to create an equivalent `Structure` using Cartesian coordinates:

In [54]:
bcc_fe_from_cart = Structure(Lattice.cubic(2.8), ["Fe", "Fe"], [[0, 0, 0], [1.4, 1.4, 1.4]],
                             coords_are_cartesian=True)

In [55]:
print(bcc_fe_from_cart)

Full Formula (Fe2)
Reduced Formula: Fe
abc   :   2.800000   2.800000   2.800000
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0.5  0.5  0.5


We can check that both structures are equivalent:

In [56]:
bcc_fe == bcc_fe_from_cart

True

As in a `Molecule`, we can access properties of the structure, such as its volume:

In [57]:
bcc_fe.volume

21.951999999999995

## 2.2.1 Creating Structures from Spacegroups

Structures can also be created directly from their spacegroup:

In [58]:
bcc_fe = Structure.from_spacegroup("Im-3m", Lattice.cubic(2.8), ["Fe"], [[0, 0, 0]])
print(bcc_fe)

Full Formula (Fe2)
Reduced Formula: Fe
abc   :   2.800000   2.800000   2.800000
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0.5  0.5  0.5


In [59]:
nacl = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5.692), ["Na+", "Cl-"],
                                 [[0, 0, 0], [0.5, 0.5, 0.5]])
nacl

Structure Summary
Lattice
    abc : 5.692 5.692 5.692
 angles : 90.0 90.0 90.0
 volume : 184.41433388800002
      A : 5.692 0.0 0.0
      B : 0.0 5.692 0.0
      C : 0.0 0.0 5.692
PeriodicSite: Na+ (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Na+ (0.0000, 2.8460, 2.8460) [0.0000, 0.5000, 0.5000]
PeriodicSite: Na+ (2.8460, 0.0000, 2.8460) [0.5000, 0.0000, 0.5000]
PeriodicSite: Na+ (2.8460, 2.8460, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Cl- (2.8460, 2.8460, 2.8460) [0.5000, 0.5000, 0.5000]
PeriodicSite: Cl- (2.8460, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Cl- (0.0000, 2.8460, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: Cl- (0.0000, 0.0000, 2.8460) [0.0000, 0.0000, 0.5000]


Structure Summary
Lattice
    abc : 5.692 5.692 5.692
 angles : 90.0 90.0 90.0
 volume : 184.41433388800002
      A : 5.692 0.0 0.0
      B : 0.0 5.692 0.0
      C : 0.0 0.0 5.692
PeriodicSite: Na+ (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Na+ (0.0000, 2.8460, 2.8460) [0.0000, 0.5000, 0.5000]
PeriodicSite: Na+ (2.8460, 0.0000, 2.8460) [0.5000, 0.0000, 0.5000]
PeriodicSite: Na+ (2.8460, 2.8460, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Cl- (2.8460, 2.8460, 2.8460) [0.5000, 0.5000, 0.5000]
PeriodicSite: Cl- (2.8460, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Cl- (0.0000, 2.8460, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: Cl- (0.0000, 0.0000, 2.8460) [0.0000, 0.0000, 0.5000]

And spacegroups can be obtained from a structure:

In [60]:
nacl.get_space_group_info()

('Fm-3m', 225)

Where 225 is the spacegroup number.

# 2.2.2 Supercells

Alright, we are now well-versed in the art of creating singular structures. But in some cases, you really don't just want one unit cell; you want a supercell. Pymatgen provides a very simple interface to create superstructures. Let's start with the simplest structure that we can imagine: Polonium, a simple cubic metal.

In [61]:
polonium = Structure(Lattice.cubic(3.4), ["Po"], [[0.0, 0.0, 0.0]])

polonium

Structure Summary
Lattice
    abc : 3.4 3.4 3.4
 angles : 90.0 90.0 90.0
 volume : 39.303999999999995
      A : 3.4 0.0 0.0
      B : 0.0 3.4 0.0
      C : 0.0 0.0 3.4
PeriodicSite: Po (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]


Structure Summary
Lattice
    abc : 3.4 3.4 3.4
 angles : 90.0 90.0 90.0
 volume : 39.303999999999995
      A : 3.4 0.0 0.0
      B : 0.0 3.4 0.0
      C : 0.0 0.0 3.4
PeriodicSite: Po (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

To make a supercell, we can just multiply the structure by a tuple!

In [62]:
supercell = polonium * (2, 2, 2)
supercell

Structure Summary
Lattice
    abc : 6.8 6.8 6.8
 angles : 90.0 90.0 90.0
 volume : 314.43199999999996
      A : 6.8 0.0 0.0
      B : 0.0 6.8 0.0
      C : 0.0 0.0 6.8
PeriodicSite: Po (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Po (0.0000, 0.0000, 3.4000) [0.0000, 0.0000, 0.5000]
PeriodicSite: Po (0.0000, 3.4000, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: Po (0.0000, 3.4000, 3.4000) [0.0000, 0.5000, 0.5000]
PeriodicSite: Po (3.4000, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Po (3.4000, 0.0000, 3.4000) [0.5000, 0.0000, 0.5000]
PeriodicSite: Po (3.4000, 3.4000, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Po (3.4000, 3.4000, 3.4000) [0.5000, 0.5000, 0.5000]


Structure Summary
Lattice
    abc : 6.8 6.8 6.8
 angles : 90.0 90.0 90.0
 volume : 314.43199999999996
      A : 6.8 0.0 0.0
      B : 0.0 6.8 0.0
      C : 0.0 0.0 6.8
PeriodicSite: Po (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Po (0.0000, 0.0000, 3.4000) [0.0000, 0.0000, 0.5000]
PeriodicSite: Po (0.0000, 3.4000, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: Po (0.0000, 3.4000, 3.4000) [0.0000, 0.5000, 0.5000]
PeriodicSite: Po (3.4000, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Po (3.4000, 0.0000, 3.4000) [0.5000, 0.0000, 0.5000]
PeriodicSite: Po (3.4000, 3.4000, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Po (3.4000, 3.4000, 3.4000) [0.5000, 0.5000, 0.5000]

# 2.2.3 Exercise: Barium Titanate

Load BaTiO3 from the CIF file provided (`BaTiO3.cif`). Replace the barium with a new element with atomic number equal to the the day of the month you were born (e.g. 1st = hydrogen, 31st = gallium). Then make a supercell of N unit cells in one cartesian direction of your choice where N is the integer of your birth month (e.g. January = 1, December = 12).

In [63]:
BaTiO3=Structure.from_file(os.path.join(data_dir, "pymatgen", "BaTiO3.cif"))
BaTiO3.replace(0,'Mg')
BaTiO3=BaTiO3*(1,1,4)
BaTiO3

Structure Summary
Lattice
    abc : 4.07715918 4.07715918 16.30863668
 angles : 89.69902153 89.69902161000002 89.6990215
 volume : 271.09098694956924
      A : 4.077102926177598 0.0 0.021417479131362785
      B : 0.021305274015716264 4.077047259439262 0.021417484824072987
      C : 0.0 0.0 16.30863668
PeriodicSite: Mg (2.0375, 2.0269, 2.0483) [0.4972, 0.4972, 0.1243]
PeriodicSite: Mg (2.0375, 2.0269, 6.1254) [0.4972, 0.4972, 0.3743]
PeriodicSite: Mg (2.0375, 2.0269, 10.2026) [0.4972, 0.4972, 0.6243]
PeriodicSite: Mg (2.0375, 2.0269, 14.2798) [0.4972, 0.4972, 0.8743]
PeriodicSite: Ti (4.0255, 4.0045, 4.0467) [0.9822, 0.9822, 0.2456]
PeriodicSite: Ti (4.0255, 4.0045, 8.1239) [0.9822, 0.9822, 0.4956]
PeriodicSite: Ti (4.0255, 4.0045, 12.2010) [0.9822, 0.9822, 0.7456]
PeriodicSite: Ti (4.0255, 4.0045, 16.2782) [0.9822, 0.9822, 0.9956]
PeriodicSite: O (2.1369, 0.0519, 0.0634) [0.5241, 0.0127, 0.0032]
PeriodicSite: O (2.1369, 0.0519, 4.1406) [0.5241, 0.0127, 0.2532]
PeriodicSite: O (2.1369, 

Structure Summary
Lattice
    abc : 4.07715918 4.07715918 16.30863668
 angles : 89.69902153 89.69902161000002 89.6990215
 volume : 271.09098694956924
      A : 4.077102926177598 0.0 0.021417479131362785
      B : 0.021305274015716264 4.077047259439262 0.021417484824072987
      C : 0.0 0.0 16.30863668
PeriodicSite: Mg (2.0375, 2.0269, 2.0483) [0.4972, 0.4972, 0.1243]
PeriodicSite: Mg (2.0375, 2.0269, 6.1254) [0.4972, 0.4972, 0.3743]
PeriodicSite: Mg (2.0375, 2.0269, 10.2026) [0.4972, 0.4972, 0.6243]
PeriodicSite: Mg (2.0375, 2.0269, 14.2798) [0.4972, 0.4972, 0.8743]
PeriodicSite: Ti (4.0255, 4.0045, 4.0467) [0.9822, 0.9822, 0.2456]
PeriodicSite: Ti (4.0255, 4.0045, 8.1239) [0.9822, 0.9822, 0.4956]
PeriodicSite: Ti (4.0255, 4.0045, 12.2010) [0.9822, 0.9822, 0.7456]
PeriodicSite: Ti (4.0255, 4.0045, 16.2782) [0.9822, 0.9822, 0.9956]
PeriodicSite: O (2.1369, 0.0519, 0.0634) [0.5241, 0.0127, 0.0032]
PeriodicSite: O (2.1369, 0.0519, 4.1406) [0.5241, 0.0127, 0.2532]
PeriodicSite: O (2.1369, 

# Summary

This notebook is intended to provide a short introduction to some of the functionality of pymatgen. We've examined the building blocks of pymatgen: the `Structure` and `Molecule` objects, the `Lattice`, `Element`, `Specie` and `Composition` objects that they're made up of, a way to visualize `Structure` and `Molecule` objects, simple `Structure` transformations, and some basic input/output functionality.